In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

Q: if they are elected, they have the metrics, if not elected, no metrics?
should I just check elected validators not all the validators?

challenge: needs rounding for the manually calcualted number, otherwise sometimes it will fail. 

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getEposMedian():
    method = "hmy_getMedianRawStakeSnapshot"
    params = []
    return get_information(method, params)['result']['epos-median-stake']

In [4]:
def getAllElectedValidator():
    method = "hmy_getElectedValidatorAddresses"
    params =[]
    return get_information(method, params)['result']

In [5]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [6]:
def M5_test():
    
    flag = True
    # get the median stake and the upper and lower level 
    median = float(getEposMedian())
    lower = median- 0.15*median
    upper = median + 0.15*median
    print("median stake is " + str(median))
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    
    # check all the elected validator's effective stake
    elected = getAllElectedValidator()
    for i in elected:
        for s in getValidatorInfo(i)['metrics']['by-shard']:
            if float(s['effective-stake']) > upper or float(s['effective-stake']) < lower:
                    print("validator: " + i + " bls public key: " + str(s['bls-public-key']) +\
                          " shard: " + str(s['shard-id'])) 
                    print("effective stake is out of range. The effective stake is " + s['effective-stake'])
                    print("")
                    flag = False
    if flag:
        return True
    else:
        return False

In [7]:
M5_test()

median stake is 2.72e+23
lower bond is 2.312e+23
upper bond is 3.128e+23


True